In [1]:
import pandas as pd
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm


tqdm.pandas()
test_data = pd.read_csv('drugsComTest_raw.tsv',sep = '\t')
train_data = pd.read_csv('drugsComTrain_raw.tsv',sep = '\t')
df_total = pd.concat([train_data, test_data], axis=0)
def drug_info(drug_name):
    # URL de la page de recherche de Drugs.com
    search_url = f"https://www.drugs.com/search.php?searchterm={drug_name.replace(' ', '+')}"
    
    # Effectuer la requête de recherche
    try:
        response = requests.get(search_url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erreur lors de la requête de recherche : {e}")
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")

    # Rechercher le premier lien
    first_link_tag = soup.select_one("div.ddc-media-content a.ddc-search-result-link-wrap")
    
    if first_link_tag and first_link_tag['href']:
        first_link = first_link_tag['href']
        
        # S'assurer que l'URL est absolue
        if not first_link.startswith("http"):
            first_link = "https://www.drugs.com" + first_link
        
        try:
            # Obtenir le contenu de la première page trouvée
            response = requests.get(first_link)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Erreur lors de la requête de la page du médicament : {e}")
            return None
        
        drug_page = BeautifulSoup(response.text, "html.parser")
        
        # Extraire des informations supplémentaires si nécessaire
        drug_info = drug_page.get_text()
        return drug_info
    else:
        print("Aucun lien trouvé pour ce médicament.")
        return None